## Identify relevenat tweets 
### The goal of this project is to analyse tweets related to resolutio 2020 and planet. in this work book we use topic analysis to identify relevant tweets for further analysis

In [ ]:
#get the libraries

import pandas as pd
from collections import OrderedDict
import requests
import boto3 
comprehend = boto3.client('comprehend', region_name='eu-west-1')
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models.keyedvectors import KeyedVectors
from gensim.corpora.textcorpus import TextCorpus
from gensim.test.utils import datapath, get_tmpfile
from gensim.similarities import Similarity
import scipy
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize

In [4]:
import sys
!{sys.executable} -m pip install gensim

You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [5]:
# Read the csv file as dataframe
srcdata = pd.read_csv('clean_text_extended.csv')
srcdf = pd.DataFrame(srcdata)
srclist = []
notepad_token = []

In [6]:
srcdf.head(3)


,Unnamed: 0,tweet_id,user_id,text,clean_tweet,clean_text_nostem
0,0,1240035700722073601,3100904593,who are you during covid19? i find myself even...,covid find even inspir work unrel paper civils...,covid find even inspired work unrelated paper ...
1,1,1240035732036571138,1237702374808051712,do you think direct cash payments to americans...,think direct cash payment american good way so...,think direct cash payments americans good way ...
2,2,1240006616759451648,74160470,the grift continues. thanks gop,grift continu thank gop,grift continues thanks gop


In [7]:
# create glove model for twitter specifically
import boto3
from sagemaker import get_execution_role
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models.keyedvectors import KeyedVectors

#method to access files in S3 for generic glove model
role = get_execution_role()
bucket='tweet-general'
data_key = 'small_tw_glove/glove_twitter_27B_50d.txt'
data_location = 's3://{}/{}'.format(bucket, data_key)

glove2word2vec(glove_input_file=data_location, word2vec_output_file="gensim_glove_tweet_vectors.txt")
glove_model = KeyedVectors.load_word2vec_format("gensim_glove_tweet_vectors.txt", binary=False)

In [51]:
#testing wordvec

glove_model.most_similar(positive=['hate'])


[('stupid', 0.8922922611236572),
 ('people', 0.8883591890335083),
 ('why', 0.8878941535949707),
 ('think', 0.8827831149101257),
 ("n't", 0.8805513978004456),
 ('reason', 0.8791501522064209),
 ('fuck', 0.8789308667182922),
 ('dont', 0.8750595450401306),
 ('swear', 0.8726238012313843),
 ('talking', 0.8694974780082703)]

In [52]:
# pretty print function
def pp(obj):
    print(pd.DataFrame(obj))
    
def analogy(worda, wordb, wordc):
    result = glove_model.most_similar(negative=[worda], 
                                positive=[wordb, wordc])
    return result[0][0]

countries = ['australia', 'canada', 'germany', 'slovakia', 'italy']
foods = [analogy('us', 'hamburger', country) for country in countries]
pp(zip(countries,foods))

           0             1
0  australia  hopman      
1  canada     cheeseburger
2  germany    lasagne     
3  slovakia   fise        
4  italy      bolognese   


In [9]:
import nltk
nltk.download('punkt')
full_srclist = srcdf["clean_tweet"].to_list()
print(len(full_srclist))

[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
39513


In [1]:
#srclist = full_srclist

In [24]:
#handling of key errors
from array import array 
#from sklearn.metrics.pairwise import cosine_similarity
#srclist = full_srclist[0:1]
srclist = full_srclist
#print(srclist)
#wordlist= ['planet', 'environment', 'nature', 'vegan', 'warming', 'green', 'ethical', 'climate', 'sustainable', 'recycling']
#wordlist for corona
wordlist = ['distancing', 'government', 'medical','shortage', 'confirmed', 'response', 'buying', 'lockdown']
#wordlist= ['planet', 'environment']
for word in wordlist:
    collist = []
    sclist = []
#Create 300d vector for each word in wordlist
    wordvec=glove_model[word]
    #np.ravel(wordvec)
    #wordvecavg = np.mean(wordvec)
    for i in srclist:
        #print(i)
        #tokenise each row of text
        try:
            notepad_token = word_tokenize(str(i))
        except:
            notepad_token = word_tokenize(' ')
        #print(notepad_token)
        wdveclist = []
        for wd in notepad_token:
        #Create 300d vector for each word in a row of text. Store the vectors in a list  
            try:
                wdvec = glove_model[wd]
                #print(wdvec)
            except:
                #KeyError
                wdvec = np.zeros(50)
            wdveclist.append(wdvec)
            #print("wordveclist", wdveclist)
        #convert the list of vectors for each notepad row to an array. Caluclate the mean of the array
        wdveclistavg = np.mean(wdveclist,axis=0)
        
        #calculate the similarity score between the keyword vector and the calculated mean
        simscore = 1 - scipy.spatial.distance.cosine(wordvec,wdveclistavg)
        if simscore is None:
            simscore = 0
        else:
            simscore = simscore    
    
     
        #print("get cosine of wordvec", wordvec, "and wdlistavg", wdveclistavg )
    
    
        #simscore = 1 - scipy.spatial.distance.cosine(wordvec,wdveclistavg)
        collist.append(simscore)
        #print("collist", collist)

#         #using scikitlearn
# #         skscore = cosine_similarity(wordvec,wdveclistavg)
# #         print(skscore)
# #         sclist.append(skscore)
# #         print("scikit", sclist)

        
#         #srcdf[word]=collist
    srcdf[word]=collist


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/scipy/spatial/distance.py:698: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [25]:
srcdf.head(10)

,Unnamed: 0,tweet_id,user_id,text,clean_tweet,clean_text_nostem,distancing,government,coping,death,response,buying,paper,lockdown,medical,shortage,confirmed
0,0,1240035700722073601,3100904593,"who are you during covid19? i find myself even more inspired to work on my (unrelated) paper on civilsociety and environmentalcrisis, since the role of civil society in this pandemic has not been well discussed (understandbly tho). \ni also picked up chess and lost twice 😷😢 tkd8cviujn",covid find even inspir work unrel paper civilsocieti environmentalcrisi sinc role civil societi pandem well discuss understandbl tho also pick chess lost twice 😷😢 tkdcviujn,covid find even inspired work unrelated paper civilsociety environmentalcrisis since role civil society pandemic well discussed understandbly tho also picked chess lost twice 😷😢 tkdcviujn,0.265822,0.669127,0.429225,0.757928,0.715828,0.693746,0.746818,0.462949,0.570855,0.427443,0.626218
1,1,1240035732036571138,1237702374808051712,do you think direct cash payments to americans is a good way to soften the economic blow of covid-19?\n\ntrump covid2019 coronavirus ubi,think direct cash payment american good way soften econom blow covid trump covid coronaviru ubi,think direct cash payments americans good way soften economic blow covid trump covid coronavirus ubi,0.255193,0.721280,0.365535,0.716188,0.686933,0.753213,0.707936,0.399542,0.577512,0.498241,0.560460
2,2,1240006616759451648,74160470,the grift continues. thanks gop,grift continu thank gop,grift continues thanks gop,0.155846,0.713355,0.346856,0.511601,0.669269,0.491507,0.464411,0.403721,0.415026,0.369650,0.487741
3,3,1246161385387515904,800890073059381248,"“5 of the 6 people around the small table for a few hours at that dinner got sick.”\n\nthat sixth person who felt fine could have spread it to so many others. stay home. stop visiting people. it sucks, but it’ll keep you and the ones you love safe.",“ peopl around small tabl hour dinner got sick ” sixth person felt fine could spread mani other stay home stop visit peopl suck it’ll keep one love safe,“ people around small table hours dinner got sick ” sixth person felt fine could spread many others stay home stop visiting people sucks it’ll keep ones love safe,0.204224,0.609084,0.309225,0.764854,0.662714,0.699547,0.679046,0.370438,0.488427,0.346516,0.584272
4,4,1246164164260827136,1079598822601510913,are you affected by the coronavirus,affect coronaviru,affected coronavirus,0.197378,0.606063,0.525567,0.509795,0.603822,0.484087,0.382019,0.366448,0.512630,0.513130,0.413338
5,5,1246162433657503748,2153079480,uk govt: provide ppe to all frontline nhs to keep them safe coronavirus - sign the petition! twx18kbenf via ukchange,uk govt provid ppe frontlin nh keep safe coronaviru sign petit twxkbenf via ukchang,uk govt provide ppe frontline nhs keep safe coronavirus sign petition twxkbenf via ukchange,0.184160,0.763694,0.279897,0.683488,0.710007,0.640997,0.647567,0.395103,0.671007,0.427271,0.614550
6,6,1246162443962920967,22084332,"😶😶what, what, \nwhat the fuck??!!😡",😶😶what fuck 😡,😶😶what fuck 😡,0.074233,0.458131,0.196292,0.626579,0.518587,0.550837,0.574005,0.372410,0.327176,0.217202,0.365901
7,7,1246162465911701504,18632386,"thank you west park staff, including food services workers like chris who are working tirelessly to ensure our patients are nourished. let’s cheer4healthworkers tonight at 7:30 risingup flattenthecurve covid19 supportourheroes heroesinscrubs men8e9rx5c",thank west park staff includ food servic worker like chri work tirelessli ensur patient nourish let’ cheerhealthwork tonight risingup flattenthecurv covid supportourhero heroesinscrub menerxc,thank west park staff including food services workers like chris working tirelessly ensure patients nourished let’s cheerhealthworkers tonight risingup flattenthecurve covid supportourheroes heroesinscrubs menerxc,0.192648,0.702301,0.366774,0.762484,0.693881,0.700546,0.642714,0.445337,0.669851,0.464116,0.606570
8,8,1246162473541144576,

In [28]:
corona_topics = srcdf[['tweet_id', 'text', 'distancing', 'government', 'medical','shortage', 'confirmed', 'response', 'buying', 'lockdown']].copy() 

In [37]:
corona_topics['main_topic'] = corona_topics[['distancing', 'government', 'medical','shortage', 'confirmed', 'response', 'buying', 'lockdown']].idxmax(axis=1)

In [22]:
pd.set_option('display.max_colwidth', -1)

In [39]:
corona_topics.head(10)

,tweet_id,text,distancing,government,medical,shortage,confirmed,response,buying,lockdown,main_topic,main_topic_value
0,1240035700722073601,"who are you during covid19? i find myself even more inspired to work on my (unrelated) paper on civilsociety and environmentalcrisis, since the role of civil society in this pandemic has not been well discussed (understandbly tho). \ni also picked up chess and lost twice 😷😢 tkd8cviujn",0.265822,0.669127,0.570855,0.427443,0.626218,0.715828,0.693746,0.462949,response,0.715828
1,1240035732036571138,do you think direct cash payments to americans is a good way to soften the economic blow of covid-19?\n\ntrump covid2019 coronavirus ubi,0.255193,0.721280,0.577512,0.498241,0.560460,0.686933,0.753213,0.399542,buying,0.753213
2,1240006616759451648,the grift continues. thanks gop,0.155846,0.713355,0.415026,0.369650,0.487741,0.669269,0.491507,0.403721,government,0.713355
3,1246161385387515904,"“5 of the 6 people around the small table for a few hours at that dinner got sick.”\n\nthat sixth person who felt fine could have spread it to so many others. stay home. stop visiting people. it sucks, but it’ll keep you and the ones you love safe.",0.204224,0.609084,0.488427,0.346516,0.584272,0.662714,0.699547,0.370438,buying,0.699547
4,1246164164260827136,are you affected by the coronavirus,0.197378,0.606063,0.512630,0.513130,0.413338,0.603822,0.484087,0.366448,government,0.606063
5,1246162433657503748,uk govt: provide ppe to all frontline nhs to keep them safe coronavirus - sign the petition! twx18kbenf via ukchange,0.184160,0.763694,0.671007,0.427271,0.614550,0.710007,0.640997,0.395103,government,0.763694
6,1246162443962920967,"😶😶what, what, \nwhat the fuck??!!😡",0.074233,0.458131,0.327176,0.217202,0.365901,0.518587,0.550837,0.372410,buying,0.550837
7,1246162465911701504,"thank you west park staff, including food services workers like chris who are working tirelessly to ensure our patients are nourished. let’s cheer4healthworkers tonight at 7:30 risingup flattenthecurve covid19 supportourheroes heroesinscrubs men8e9rx5c",0.192648,0.702301,0.669851,0.464116,0.606570,0.693881,0.700546,0.445337,government,0.702301
8,1246162473541144576,"hello my darlings, how is everyone coping with this madness. quarentinelife covid2019",0.250019,0.398735,0.268005,0.168103,0.450798,0.549012,0.559403,0.268657,buying,0.559403
9,1246162483762679809,the media has lost all of its credibility.,0.130486,0.695699,0.572919,0.334277,0.601589,0.680278,0.611628,0.333011,government,0.695699


In [35]:
corona_topics['main_topic_value'] = corona_topics['main_topic'] = corona_topics[['distancing', 'government', 'medical','shortage', 'confirmed', 'response', 'buying', 'lockdown']].max(axis=1)

In [36]:
corona_topics.head(10)

,tweet_id,text,distancing,government,medical,shortage,confirmed,response,buying,lockdown,main_topic,main_topic_value
0,1240035700722073601,"who are you during covid19? i find myself even more inspired to work on my (unrelated) paper on civilsociety and environmentalcrisis, since the role of civil society in this pandemic has not been well discussed (understandbly tho). \ni also picked up chess and lost twice 😷😢 tkd8cviujn",0.265822,0.669127,0.570855,0.427443,0.626218,0.715828,0.693746,0.462949,0.715828,0.715828
1,1240035732036571138,do you think direct cash payments to americans is a good way to soften the economic blow of covid-19?\n\ntrump covid2019 coronavirus ubi,0.255193,0.721280,0.577512,0.498241,0.560460,0.686933,0.753213,0.399542,0.753213,0.753213
2,1240006616759451648,the grift continues. thanks gop,0.155846,0.713355,0.415026,0.369650,0.487741,0.669269,0.491507,0.403721,0.713355,0.713355
3,1246161385387515904,"“5 of the 6 people around the small table for a few hours at that dinner got sick.”\n\nthat sixth person who felt fine could have spread it to so many others. stay home. stop visiting people. it sucks, but it’ll keep you and the ones you love safe.",0.204224,0.609084,0.488427,0.346516,0.584272,0.662714,0.699547,0.370438,0.699547,0.699547
4,1246164164260827136,are you affected by the coronavirus,0.197378,0.606063,0.512630,0.513130,0.413338,0.603822,0.484087,0.366448,0.606063,0.606063
5,1246162433657503748,uk govt: provide ppe to all frontline nhs to keep them safe coronavirus - sign the petition! twx18kbenf via ukchange,0.184160,0.763694,0.671007,0.427271,0.614550,0.710007,0.640997,0.395103,0.763694,0.763694
6,1246162443962920967,"😶😶what, what, \nwhat the fuck??!!😡",0.074233,0.458131,0.327176,0.217202,0.365901,0.518587,0.550837,0.372410,0.550837,0.550837
7,1246162465911701504,"thank you west park staff, including food services workers like chris who are working tirelessly to ensure our patients are nourished. let’s cheer4healthworkers tonight at 7:30 risingup flattenthecurve covid19 supportourheroes heroesinscrubs men8e9rx5c",0.192648,0.702301,0.669851,0.464116,0.606570,0.693881,0.700546,0.445337,0.702301,0.702301
8,1246162473541144576,"hello my darlings, how is everyone coping with this madness. quarentinelife covid2019",0.250019,0.398735,0.268005,0.168103,0.450798,0.549012,0.559403,0.268657,0.559403,0.559403
9,1246162483762679809,the media has lost all of its credibility.,0.130486,0.695699,0.572919,0.334277,0.601589,0.680278,0.611628,0.333011,0.695699,0.695699


In [126]:
from array import array 
#from sklearn.metrics.pairwise import cosine_similarity
#srclist = full_srclist[0:1]
srclist = full_srclist
#print(srclist)
wordlist= ['planet', 'environment', 'nature', 'vegan', 'warming', 'green', 'ethical', 'climate', 'sustainable']
#wordlist= ['planet', 'environment']
for word in wordlist:
    collist = []
    sclist = []
    print(word)
#Create 300d vector for each word in wordlist
    wordvec=glove_model[word]
    #np.ravel(wordvec)
    #wordvecavg = np.mean(wordvec)
    for i in srclist:
        #print(i)
        #tokenise each row of text
        try:
            notepad_token = word_tokenize(str(i))
        except:
            notepad_token = word_tokenize(' ')
        #print(notepad_token)
        wdveclist = []
        for wd in notepad_token:
        #Create 300d vector for each word in a row of text. Store the vectors in a list  
            try:
                wdvec = glove_model[wd]
            #print(wdveclist)
            except:
                wdvec = np.zeros(50)
            wdveclist.append(wdvec)
            #print("wordveclist", wdveclist)
        #convert the list of vectors for each notepad row to an array. Caluclate the mean of the array
        #wdveclistarr = np.array(wdveclist)
        wdveclistavg = np.mean(wdveclist,axis=0)
        #print(wdveclistavg)
        wdveclistweightedavg = np.average(wdveclist)
#       #calculate the similarity score between the keyword vector and the calculated mean
        #print("get cosine of wordvec", wordvec, "and wdlistavg", wdveclistavg )
        simscore = 1 - scipy.spatial.distance.cosine(wordvec,wdveclistavg)
        collist.append(simscore)
    #print("collist", collist)

        #using scikitlearn
#         skscore = cosine_similarity(wordvec,wdveclistavg)
#         print(skscore)
#         sclist.append(skscore)
#         print("scikit", sclist)

        
        #srcdf[word]=collist
    srcdf[word]=collist


planet


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/scipy/spatial/distance.py:698: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


environment
nature
vegan
warming
green
ethical
climate
sustainable


In [129]:
srcdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10459 entries, 0 to 10458
Data columns (total 19 columns):
tweet_id               10459 non-null object
user_id                10459 non-null object
text                   10459 non-null object
clean_text             10458 non-null object
tweet_language         10458 non-null object
tweet_language_full    10458 non-null object
translated_text        10458 non-null object
key_phrases            10344 non-null object
Sentiment              10457 non-null object
planet                 9974 non-null float64
environment            9974 non-null float64
nature                 9974 non-null float64
vegan                  9974 non-null float64
warming                9974 non-null float64
green                  9974 non-null float64
ethical                9974 non-null float64
climate                9974 non-null float64
sustainable            9974 non-null float64
recycling              9974 non-null float64
dtypes: float64(10), object(9)
memor

In [97]:
#testing null values
#test1 = glove_model['nextravagant']
#wordlist= ['planet', 'environment', 'nature', 'vegan', 'warming', 'green', 'ethical', 'climate', 'sustainable', 'jruoewhgeuir']
wordlist = ['2020', '2020', 'perfect', 'vision', 'sure', 'vision', 'newyear2020', 'eye', 'xxxxxx']

for i in wordlist:
    try:
        result1 = glove_model[i]
        result = np.mean(result1)
        
    except:
        KeyError
        result = 0
    print(i,result1)

#print(np.mean(0))
# try:
#     result = glove_model[word]
# except: 
#     KeyError
#     result = '0'
# print (result)


2020 [-7.1077e-01 -3.5075e-01 -1.9972e-01  1.3835e-01 -3.4037e-01 -4.1139e-01
  9.9436e-01 -2.6643e-01 -2.6639e-01  6.0467e-01  8.0660e-02 -3.7310e-03
 -3.7981e+00 -2.1132e-02 -5.9648e-01 -1.3985e-01  3.8573e-01 -1.8900e-01
  6.1866e-01 -1.0225e+00 -1.1350e+00  1.3375e-01  4.3337e-01 -1.5024e-02
 -1.5243e-02  5.8293e-01 -8.6330e-01  7.0792e-01 -2.6590e-01  5.4302e-01
 -5.4548e-01 -9.8515e-02  1.1407e-01 -1.0229e+00  4.4107e-01  1.4034e-01
 -5.9349e-01  4.3542e-01  2.7181e-01  5.5835e-01 -4.8260e-01 -1.4403e-01
  1.3827e-01  6.2529e-01  4.1560e-01 -4.2782e-01  7.0738e-01  1.9695e-03
  5.1502e-01  3.9076e-01]
2020 [-7.1077e-01 -3.5075e-01 -1.9972e-01  1.3835e-01 -3.4037e-01 -4.1139e-01
  9.9436e-01 -2.6643e-01 -2.6639e-01  6.0467e-01  8.0660e-02 -3.7310e-03
 -3.7981e+00 -2.1132e-02 -5.9648e-01 -1.3985e-01  3.8573e-01 -1.8900e-01
  6.1866e-01 -1.0225e+00 -1.1350e+00  1.3375e-01  4.3337e-01 -1.5024e-02
 -1.5243e-02  5.8293e-01 -8.6330e-01  7.0792e-01 -2.6590e-01  5.4302e-01
 -5.4548e-01 -9

In [103]:
srcdf

,tweet_id,user_id,text,clean_text,tweet_language,tweet_language_full,translated_text,key_phrases,Sentiment,planet,environment,nature,vegan,warming,green,ethical,climate,sustainable
0,1212302441603055600,1106537442197364700,welcome 2020 with 20/20! perfect vision with sure vision!!\nnewyear2020 surevision eye care happynewyear resolution2020 lukvxydkx8,welcome 2020 2020 perfect vision sure vision newyear2020 surevision eye care happynewyear resolution2020 lukvxydkx8,en,English,welcome 2020 2020 perfect vision sure vision newyear2020 surevision eye care happynewyear resolution2020 lukvxydkx8,2020 2020 perfect vision sure vision newyear2020 eye,NEUTRAL,0.652793,0.629237,0.761745,0.372245,0.625190,0.713165,0.438898,0.537456,0.450677
1,1212319183972847600,702628629814124500,"happy newyear, friends! i think the simplest of messages are always the most profound. if this image was the only thing you thought every day ... you already won ‼️ 👍😉newyears wednesdaywisdom wednesdaymotivation wednesdaythoughts wednesdayvibes fb ff resolution2020 fvwdjzwz04",happy newyear friends think simplest messages always profound image thing thought every day already won newyears wednesdaywisdom wednesdaymotivation wednesdaythoughts wednesdayvibes fb ff resolution2020 fvwdjzwz04,en,English,happy newyear friends think simplest messages always profound image thing thought every day already won newyears wednesdaywisdom wednesdaymotivation wednesdaythoughts wednesdayvibes fb ff resolution2020 fvwdjzwz04,happy newyear friends simplest messages profound image thing every day wednesdaywisdom wednesdaymotivation wednesdaythoughts,POSITIVE,0.661784,0.575906,0.669689,0.457618,0.549844,0.616016,0.412713,0.484342,0.430556
2,1212316352238751700,2810768636,حشر میں اے خدا، نہ لے مجھ سے حساب ِخیر و شر \nمیں نے تو سب بُھلا دیا رات کا خواب جان کے \n\nresolution2020,حشر میں اے خدا نہ لے مجھ سے حساب خیر و شر میں نے تو سب بھلا دیا رات کا خواب جان کے resolution2020,ur,Urdu,"O God, do not take account of me. I have forgotten all the night's dream resolution2020",O God account all the night,NEGATIVE,0.649220,0.521676,0.570661,0.417451,0.576328,0.704301,0.295537,0.452660,0.338595
3,1212028647030296600,229207628,+1 newyearchallenge resolution2020 ahzbebkksm,1 newyearchallenge resolution2020 ahzbebkksm,en,English,1 newyearchallenge resolution2020 ahzbebkksm,1,NEUTRAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1213480248333000700,2568136825,"hiit workouts help you get fitter, feel stronger and be healthier faster: olrk1mrll7 resolution2020 resolutions fitnessjourney healthy lifestyle weightloss giftideas giftsforher giftsforhim from darebees autopost sponsor prevail … 6pbflz2jpa",hiit workouts help get fitter feel stronger healthier faster olrk1mrll7 resolution2020 resolutions fitnessjourney healthy lifestyle weightloss giftideas giftsforher giftsforhim darebees autopost sponsor prevail 6pbflz2jpa,en,English,hiit workouts help get fitter feel stronger healthier faster olrk1mrll7 resolution2020 resolutions fitnessjourney healthy lifestyle weightloss giftideas giftsforher giftsforhim darebees autopost sponsor prevail 6pbflz2jpa,hiit workouts fitter olrk1mrll7 resolution2020 resolutions healthy lifestyle giftideas giftsforher giftsforhim darebees autopost,NEUTRAL,0.322674,0.664328,0.457259,0.598286,0.549528,0.408675,0.618768,0.520719,0.700318
5,1213468689485455400,189148850,while supplies last get 20% off the legendary beverly international at dps nutrition with coupon dps10! resolution2020 newyearsresolution\n\n oduhnjk0uq e6r7pzjprx,supplies last get 20 legendary beverly international dps nutrition coupon dps10 resolution2020 newyearsresolution oduhnjk0uq e6r7pzjprx,en,English,supplies last get 20 legendary beverly international dps nutrition coupon dps10 resolution2020 newyearsresolution oduhnjk0uq e6r7pzjprx,supplies 20 legendary international dps nutrition coupon,NEUTRAL,0.457161,0.660055,0.519065,0.644908,0.525719,0.663787,0.587196,0.524790,0.657183
6,1213468439332970500,189148850,*

In [134]:
#check the phrases
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)
srcdf[srcdf.tweet_id == '1212302441603055600'][['key_phrases', 'clean_text']]

,key_phrases,clean_text


In [58]:
#srcdf.loc[(srcdf['tweet_id']== '1212302441603055600' & (srcdf['key_phrases'])]
#print(srcdf[srcdf['tweet_id'] == '1212302441603055600']['key_phrases'] )
srcdf

,tweet_id,user_id,text,clean_text,tweet_language,tweet_language_full,translated_text,key_phrases,Sentiment,planet,environment,nature,vegan,warming,green,ethical,climate,sustainable
0,1212302441603055600,1106537442197364700,welcome 2020 with 20/20! perfect vision with s...,welcome 2020 2020 perfect vision sure vision ...,en,English,welcome 2020 2020 perfect vision sure vision ...,2020 2020 perfect vision sure vision newyear2...,NEUTRAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1212319183972847600,702628629814124500,"happy newyear, friends! i think the simplest o...",happy newyear friends think simplest message...,en,English,happy newyear friends think simplest message...,happy newyear friends simplest messages profo...,POSITIVE,0.661784,0.575906,0.669689,0.457618,0.549844,0.616015,0.412713,0.484342,0.430556
2,1212316352238751700,2810768636,حشر میں اے خدا، نہ لے مجھ سے حساب ِخیر و شر \n...,حشر میں اے خدا نہ لے مجھ سے حساب خیر و شر میں ...,ur,Urdu,"O God, do not take account of me. I have forgo...",O God account all the night,NEGATIVE,0.661784,0.575906,0.669689,0.457618,0.549844,0.616015,0.412713,0.484342,0.430556
3,1212028647030296600,229207628,+1 newyearchalle...,1 newyearchallenge resolution2020 ahzbebkksm,en,English,1 newyearchallenge resolution2020 ahzbebkksm,1,NEUTRAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1213480248333000700,2568136825,"hiit workouts help you get fitter, feel strong...",hiit workouts help get fitter feel stronger h...,en,English,hiit workouts help get fitter feel stronger h...,hiit workouts fitter olrk1mrll7 resolution2020...,NEUTRAL,0.322674,0.664328,0.457259,0.598286,0.549528,0.408675,0.618768,0.520719,0.700318
5,1213468689485455400,189148850,while supplies last get 20% off the legendary ...,supplies last get 20 legendary beverly intern...,en,English,supplies last get 20 legendary beverly intern...,supplies 20 legendary international dps nutri...,NEUTRAL,0.457161,0.660055,0.519065,0.644908,0.525719,0.663787,0.587196,0.524790,0.657183
6,1213468439332970500,189148850,*** while supplies last *** get a 4.09lb of op...,supplies last get 409lb optimum nutrition go...,en,English,supplies last get 409lb optimum nutrition go...,supplies 409lb optimum nutrition gold standard...,NEUTRAL,0.380324,0.625391,0.476579,0.641789,0.513208,0.668827,0.587305,0.517566,0.651379
7,1213198713642004500,758780303662014500,resolution2020 is to write every day. i took a...,resolution2020 write every day took exercise ...,en,English,resolution2020 write every day took exercise ...,resolution2020 every day exercise 3 epiphany s...,NEUTRAL,0.380324,0.625391,0.476579,0.641789,0.513208,0.668827,0.587305,0.517566,0.651379
8,1216403813332025300,381184049,explainer: what are the underlying causes of a...,explainer underlying causes australia shockin...,en,English,explainer underlying causes australia shockin...,underlying causes,NEUTRAL,0.436603,0.574829,0.541541,0.330488,0.634230,0.240695,0.622568,0.605072,0.466077
9,1211645532776517600,1113747113790332900,the knepp estate is one of the most incredible...,knepp estate one incredible wildland farming p...,en,English,knepp estate one incredible wildland farming p...,knepp estate one wildland farming projects eur...,NEUTRAL,0.436603,0.574829,0.541541,0.330488,0.634230,0.240695,0.622568,0.605072,0.466077


In [ ]:
#trying scikitlearn for comparison as the values are coming negative. also need to handle the 0

skscore = cosine_similarity(wordvec,wdveclistavg)
simscore = 1 - scipy.spatial.distance.cosine(wordvec,wdveclistavg)
        collist.append(simscore)
        print(collist)

In [104]:
srcdf.count(axis = 0) 

tweet_id               3356
user_id                3356
text                   3356
clean_text             3356
tweet_language         3356
tweet_language_full    3356
translated_text        3356
key_phrases            3315
Sentiment              3356
planet                 3154
environment            3154
nature                 3154
vegan                  3154
warming                3154
green                  3154
ethical                3154
climate                3154
sustainable            3154
dtype: int64

In [136]:
srcdf_full[['key_phrases', 'planet', 'environment']].sort_values(by=['planet'], ascending = False)


,key_phrases,planet,environment
8204,voicefortheplanet planet,1.000000,0.521601
8759,resolution2020 planet earth,0.966506,0.570205
4621,planet last generation damage planet voice planet,0.896532,0.596046
8410,stand planet,0.884745,0.565563
6343,planet series journey biodiversity planet sure human nature,0.881363,0.733140
5474,planet wwf voicefortheplanet protecttheplanet protectmotherearth naturelovers,0.877993,0.517579
1890,resolution2020 movement planet 8psutfvbno,0.867683,0.695921
3670,the wwf nature stephencorrysvl planet voicefortheplanet financialization nature,0.841974,0.699023
6844,4 resolutions2020 the planet action,0.838406,0.643950
187,planet lifetime,0.836053,0.563869


In [137]:
#df to be saved only contains the relevant columns
relevancy_df_full = srcdf_full[['tweet_id','user_id','key_phrases','Sentiment','planet','environment','nature','vegan','warming','green','ethical','climate','sustainable',
]]

relevancy_df_full.set_index('tweet_id')

,user_id,key_phrases,Sentiment,planet,environment,nature,vegan,warming,green,ethical,climate,sustainable
tweet_id,,,,,,,,,,,,
1213391474030723072,1209876717461999618,my tweetees this 1st weekend fanfare the sweet madness laughter and happiness the resolutions,POSITIVE,0.664089,0.606871,0.678213,0.486616,0.653909,0.734026,0.375480,0.539838,0.449539
1213371548012138496,884306064,enough days,POSITIVE,0.572498,0.557091,0.561274,0.416239,0.570469,0.664203,0.338669,0.485343,0.400163
1213318236357292033,1462008055,resolution2020,NEUTRAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1216403609614782465,1069033017124052992,good environment,POSITIVE,0.635743,0.840888,0.724010,0.541867,0.681774,0.689585,0.567104,0.686407,0.659969
1212293933889392643,1059734552044863489,Happy New Year all your 2020 tweets mine a good resolution,POSITIVE,0.607030,0.509481,0.591425,0.429988,0.539868,0.694881,0.354126,0.451063,0.352077
1212015867510304770,189148850,miss amazing deal ansi nutrition 3996,NEUTRAL,0.555731,0.632159,0.599154,0.544698,0.583618,0.722543,0.483065,0.541835,0.548506
1212247893853319168,29051822,rotoradarceo resolution2020 keto diet 40 lbs year,NEGATIVE,0.483009,0.484225,0.383441,0.546220,0.512391,0.602497,0.358393,0.425021,0.458297
1212820585861959680,224721210,new year new overcoming bad vices 3epqmu6ete badhabits mindovermatter,NEUTRAL,0.566850,0.615595,0.611246,0.518442,0.647793,0.651392,0.499427,0.590144,0.526483
1212712906044911617,1169547897580355586,women offwhite amp maroon straight kurta material cottonsilk togccykcao divinafashions newyear newyear2020 newyearresolution resolution2020 newyearnewme fashions myntra myntrafashion myntratrends,NEUTRAL,0.505037,0.491318,0.501586,0.525423,0.454597,0.681518,0.478044,0.402995,0.523977


In [112]:
relevancy_df.set_index('tweet_id').head(3)

,user_id,key_phrases,Sentiment,planet,environment,nature,vegan,warming,green,ethical,climate,sustainable
tweet_id,,,,,,,,,,,,
1212302441603055600,1106537442197364700,2020 2020 perfect vision sure vision newyear2020 eye,NEUTRAL,0.652793,0.629237,0.761745,0.372245,0.625190,0.713165,0.438898,0.537456,0.450677
1212319183972847600,702628629814124500,happy newyear friends simplest messages profound image thing every day wednesdaywisdom wednesdaymotivation wednesdaythoughts,POSITIVE,0.661784,0.575906,0.669689,0.457618,0.549844,0.616016,0.412713,0.484342,0.430556
1212316352238751700,2810768636,O God account all the night,NEGATIVE,0.649220,0.521676,0.570661,0.417451,0.576328,0.704301,0.295537,0.452660,0.338595


In [113]:
relevancy_df

,tweet_id,user_id,key_phrases,Sentiment,planet,environment,nature,vegan,warming,green,ethical,climate,sustainable
0,1212302441603055600,1106537442197364700,2020 2020 perfect vision sure vision newyear2020 eye,NEUTRAL,0.652793,0.629237,0.761745,0.372245,0.625190,0.713165,0.438898,0.537456,0.450677
1,1212319183972847600,702628629814124500,happy newyear friends simplest messages profound image thing every day wednesdaywisdom wednesdaymotivation wednesdaythoughts,POSITIVE,0.661784,0.575906,0.669689,0.457618,0.549844,0.616016,0.412713,0.484342,0.430556
2,1212316352238751700,2810768636,O God account all the night,NEGATIVE,0.649220,0.521676,0.570661,0.417451,0.576328,0.704301,0.295537,0.452660,0.338595
3,1212028647030296600,229207628,1,NEUTRAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1213480248333000700,2568136825,hiit workouts fitter olrk1mrll7 resolution2020 resolutions healthy lifestyle giftideas giftsforher giftsforhim darebees autopost,NEUTRAL,0.322674,0.664328,0.457259,0.598286,0.549528,0.408675,0.618768,0.520719,0.700318
5,1213468689485455400,189148850,supplies 20 legendary international dps nutrition coupon,NEUTRAL,0.457161,0.660055,0.519065,0.644908,0.525719,0.663787,0.587196,0.524790,0.657183
6,1213468439332970500,189148850,supplies 409lb optimum nutrition gold standard whey expiry jan 2020 dps nutrition coupon,NEUTRAL,0.380324,0.625391,0.476579,0.641789,0.513208,0.668827,0.587305,0.517566,0.651379
7,1213198713642004500,758780303662014500,resolution2020 every day exercise 3 epiphany short short story silent witness thanks,NEUTRAL,0.651237,0.575956,0.658506,0.474039,0.598216,0.675684,0.408619,0.502858,0.416612
8,1216403813332025300,381184049,underlying causes,NEUTRAL,0.436603,0.574829,0.541541,0.330488,0.634230,0.240695,0.622568,0.605072,0.466077
9,1211645532776517600,1113747113790332900,knepp estate one wildland farming projects europe latest video collaboration wwf evxxqvgfk0 netflix,NEUTRAL,0.636106,0.681439,0.589937,0.542994,0.674298,0.693336,0.528794,0.651922,0.641195


In [140]:
#exporting the full dataframe
from io import StringIO
import boto3

def write_pd_s3_csv(df, bucket, filepath):
    csv_buffer = StringIO()
    df.to_csv(csv_buffer, index = False)
    s3_resource = boto3.resource('s3')
    s3_resource.Object(bucket, filepath).put(Body=csv_buffer.getvalue())
    print("The data is successfully written to S3 path:", bucket+"/"+filepath)

# Write to S3
s3_bucket =  'tweet-general'  
file_path = 'relevant/qlick_relevant/relevant_full_h.csv'
tweet_text = relevancy_df_full
write_pd_s3_csv(tweet_text, s3_bucket, file_path)

The data is successfully written to S3 path: tweet-general/relevant/qlick_relevant/relevant_full_h.csv


In [11]:
srcdf.sort_values(by=['planet'])

,tweet_id,user_id,text,clean_text,tweet_language,tweet_language_full,translated_text,key_phrases,Sentiment,planet,environment,nature,vegan,warming,green,ethical,climate,sustainable
1092,1212793163401834500,101191002,fakenews \nresolution2020 7yta3rypjd,fakenews resolution2020 7yta3rypjd,en,English,fakenews resolution2020 7yta3rypjd,fakenews,NEGATIVE,-0.229541,-0.028914,-0.108376,0.018378,-0.070513,-0.178730,0.018204,0.009523,0.029967
2213,1212883351486566400,411223117,newyearsday 2020goals resolution2020 happynewy...,newyearsday 2020goals resolution2020 happynewy...,en,English,newyearsday 2020goals resolution2020 happynewy...,newyearsday 2020goals resolution2020 happynewy...,NEUTRAL,-0.203552,0.048983,-0.090306,0.127895,0.015158,-0.153462,0.019041,0.065701,0.091849
2218,1213752532536705000,103553773,huhahaha resolution2020 8kgt8jzxtm,huhahaha resolution2020 8kgt8jzxtm,tl,Tagalog,huhaha resolution2020 8kgt8jzxtm,huhaha,NEUTRAL,-0.177571,-0.121693,-0.250542,-0.140711,0.007946,-0.332107,0.093469,-0.047832,-0.097237
3219,1212263689882980400,807251229621489700,2020newyear resolution2020 morningmotivation f...,2020newyear resolution2020 morningmotivation f...,en,English,2020newyear resolution2020 morningmotivation f...,resolution2020 morningmotivation,NEUTRAL,-0.136186,0.076405,0.013298,0.220409,0.079689,-0.050914,0.099269,0.094420,0.133005
623,1211868900880736300,1188976730217832400,gfjcfdnyqc newyearseve newyearnewme resolutio...,gfjcfdnyqc newyearseve newyearnewme resolution...,en,English,gfjcfdnyqc newyearseve newyearnewme resolution...,newyearseve newyearnewme resolution2020 newyea...,NEUTRAL,-0.123159,0.081708,-0.024011,0.202051,0.055776,-0.066908,0.091401,0.092824,0.094625
601,1212340928674979800,1147977539970687000,newyearseve resolution2020 6zyondqwpw,newyearseve resolution2020 6zyondqwpw,en,English,newyearseve resolution2020 6zyondqwpw,newyearseve,NEUTRAL,-0.120204,0.066098,0.002745,0.153166,-0.051233,-0.094285,-0.050528,-0.034315,0.030657
2388,1212070581648199700,1149295576112074800,newyearchallenge newyearseve welcome2020 2020g...,newyearchallenge newyearseve welcome2020 2020g...,en,English,newyearchallenge newyearseve welcome2020 2020g...,newyearseve welcome2020 2020goals nye2020,NEUTRAL,-0.120204,0.066098,0.002745,0.153166,-0.051233,-0.094285,-0.050528,-0.034315,0.030657
2684,1212005225504157700,64189266,newyearseve tuesdaymotivation \n\nresolution20...,newyearseve tuesdaymotivation resolution2020 f...,en,English,newyearseve tuesdaymotivation resolution2020 f...,newyearseve tuesdaymotivation,NEUTRAL,-0.120204,0.066098,0.002745,0.153166,-0.051233,-0.094285,-0.050528,-0.034315,0.030657
1869,1212687185952026600,57352490,resolution2020 jpp 😂🤪👍 bburpwupkq,resolution2020 jpp bburpwupkq,id,Indonesian,resolution2020 jpp bburpwupkq,resolution2020 jpp,NEUTRAL,-0.114942,0.117143,0.091423,-0.222885,0.131067,-0.092075,-0.043999,0.139037,-0.095408
1180,1212196006361751600,3166217539,newyearseve newyears resolution2020 resolution...,newyearseve newyears resolution2020 resolution...,en,English,newyearseve newyears resolution2020 resolution...,newyearseve newyears resolution2020,NEUTRAL,-0.110663,0.101730,0.015639,0.170415,0.040755,-0.047211,0.003377,0.039983,0.073058


In [135]:
#srcdf.to_csv('tweets_glove_general.csv', index=False)

In [26]:
clean_text = srcdf[['tweet_id', 'user_id', 'clean_text', 'tweet_language', 'translated_text']].copy()

In [27]:
clean_text.set_index('tweet_id').head(3)

,user_id,clean_text,tweet_language,translated_text
tweet_id,,,,
1212711553323147264,189148850,massive sale add coupon dps10 get 50 buy ...,en,massive sale add coupon dps10 get 50 buy ...
1212049646895484931,760244020589240320,goodbye 2019 really hard deal thanks learnt g...,en,goodbye 2019 really hard deal thanks learnt g...
1216400229018284033,991674300783525888,ever considered using reclaimed wood home desi...,en,ever considered using reclaimed wood home desi...


In [44]:
# corona_topics.to_csv('corona_topics.csv')

In [46]:
# corona_main_topic = corona_topics[['tweet_id', 'main_topic', 'main_topic_value']].copy()

In [47]:
#this was reused for corona topics analysis
# from io import StringIO
# import boto3

# def write_pd_s3_csv(df, bucket, filepath):
#     csv_buffer = StringIO()
#     df.to_csv(csv_buffer, index = False)
#     s3_resource = boto3.resource('s3')
#     s3_resource.Object(bucket, filepath).put(Body=csv_buffer.getvalue())
#     print("The data is successfully written to S3 path:", bucket+"/"+filepath)

# # Write to S3
# s3_bucket =  'tweet-general'  
# file_path = 'corona/topics/main_topics.csv'
# tweet_text = corona_main_topic
# write_pd_s3_csv(tweet_text, s3_bucket, file_path)

The data is successfully written to S3 path: tweet-general/corona/topics/main_topics.csv
